# IBM Applied Data Science - Capstone Project: The Battle of Neighborhoods (peer-graded assignment)

## Introduction/Business Problem
A restaurant business company would like open an Indian restaurant in the San Francisco Bay Area, California. There are a lot of Indians working in the Bay area tech companies. Also, considerable number of Indian students are studying in the universities and colleges located in the Bay Area. Along with Indians, people from other nations also like Indian food very much. Therefore, San Francisco Bay Area has good potential to open an Indian restaurant.

San Francisco Bay Area has made up of nine counties, namely, Santa Clara, Alameda, San Mateo, San Francisco, Contra Costa, Marin, Solano, Napa, and Sonoma. There are 186 ZIP codes and 97 boroughs in the Bay Area. Since there are a large number of boroughs in the Bay Area, it is important to understand which place is the best to start a new branch of an Indian restaurant.

#### Target Audience:
Client is a restaurant business company. As a data analyst, we should help the restaurant business company that wants to start an Indian restaurant by recommending the best suitable borough and explain why this place is better than others in the Bay Area.

#### Success Criteria:
The success criteria of the project will be a good recommendation of borough in the Bay area for an Indian restaurant business.

## Data
The San Francisco Bay Area ZIP codes and PO_NAME are downloaded from DataSF open data website. The following csv file downloadable link is used to create a Pandas dataframe in Python. https://data.sfgov.org/api/views/f9wk-m4qb/rows.csv?accessType=DOWNLOAD 
From this dataframe, a new dataframe is generated with ZIP codes and PO_NAME, and then PO_NAME is renamed to ‘Borough’.

The geographical coordinates are retrieved for each borough in the dataframe using the geocoder and then leveraged the Foursquare AIP service to collect the location data, explore each borough of the Bay area and obtain information about different venue categories, especially restaurants, cafes, bars, pubs, etc. within 1000 m radius from the geographical coordinates. This data is used to understand better that how many eateries are available in the Bay area boroughs. Then the dataset is analyzed systematically to identify best suitable boroughs to open an Indian restaurant business.

## Methodology
#### Business Understanding:
The project main goal is to find optimal boroughs in the San Francisco Bay Area for a new Indian restaurant business.

#### Analytic Approach:
There are two steps to tackle this problem. In the first step, the objective is to find borough where there are a lot of eateries in the Bay area. Then these boroughs are used as “Center of Attraction” for dining with different food varieties. At the same time from these boroughs, exclude boroughs where there are Indian restaurants. This is to avoid competition with other Indian restaurants. But, it is supposed that an Indian restaurant can still attract a part of customers where there are a lot of restaurants with other kinds of dishes. Thus,
(i) Find , for example, top 25 boroughs with maximum number of restaurants, cafes, bars, pubs, etc.;
(ii) Find boroughs with Indian restaurants;
(iii) Exclude the boroughs with Indian restaurants from the top 25 borough with restaurants, cafes, etc.;
(iv) Find short-list of boroughs where there are a lot of restaurants but no Indian restaurant.
In the second step, the goal is to select clusters similar to boroughs with Indian restaurants. In any one of the boroughs where there is an Indian restaurant that continuous to work successfully, then we should find borough with similar conditions. Therefore, k-means clustering algorithm is used to find clusters.

### Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!pip install geocoder
import geocoder
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



geopy-2.1.0          | 64 KB     

San Francisco Bay area ZIP Codes URL: https://data.sfgov.org/api/views/f9wk-m4qb/rows.csv?accessType=DOWNLOAD

In [2]:
# read Bay Area ZIP code csv file by pandas
df_raw1 = pd.read_csv('https://data.sfgov.org/api/views/f9wk-m4qb/rows.csv?accessType=DOWNLOAD')
df_raw1.head()

,PO_NAME,the_geom,ZIP,STATE,Area__,Length__
0,NAPA,MULTIPOLYGON (((-122.10329200180091 38.5132829...,94558,CA,1.231326e+10,995176.225313
1,FAIRFIELD,MULTIPOLYGON (((-121.947475002335 38.301511000...,94533,CA,9.917861e+08,200772.556587
2,DIXON,MULTIPOLYGON (((-121.65335500334429 38.3133870...,95620,CA,7.236950e+09,441860.201400
3,SONOMA,MULTIPOLYGON (((-122.406843003057 38.155681999...,95476,CA,3.001414e+09,311318.546326
4,NAPA,MULTIPOLYGON (((-122.29368500225117 38.1552379...,94559,CA,1.194302e+09,359104.646602


### Choose useful columns and rename columns

In [3]:
cols = ['ZIP', 'PO_NAME']
df_raw2 = df_raw1[cols]
df_raw2 = df_raw2.rename(columns = {'PO_NAME': 'Borough'})
df_raw2

,ZIP,Borough
0,94558,NAPA
1,94533,FAIRFIELD
2,95620,DIXON
3,95476,SONOMA
4,94559,NAPA
5,94954,PETALUMA
6,94571,RIO VISTA
7,94535,TRAVIS AFB
8,94503,AMERICAN CANYON
9,94949,NOVATO


### Use python geocoder library to get geograpical coordinates

In [4]:
# get coordinates of Bay Atrea, California
bayarea_address = 'Bay Area, CA'

geolocator = Nominatim(user_agent="Bay_Area_explorer")
location = geolocator.geocode(bayarea_address)
latitude = location.latitude
longitude = location.longitude
ldn_center = [latitude, longitude]
print('The geograpical coordinate of {} are {}, {}.'.format(bayarea_address, latitude, longitude))

The geograpical coordinate of Bay Area, CA are 37.7884969, -122.3558473.


In [5]:
# define a function to get coordinates
def get_latlng(borough):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, {}'.format(borough,bayarea_address))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [get_latlng(borough) for borough in df_raw2["Borough"].tolist()]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
df_raw2['Latitude'] = df_coords['Latitude']
df_raw2['Longitude'] = df_coords['Longitude']
#print("Geographical coordinates of five borough are as below")
df_raw2

,ZIP,Borough,Latitude,Longitude
0,94558,NAPA,38.026885,-121.881924
1,94533,FAIRFIELD,38.249240,-122.044300
2,95620,DIXON,59.383710,-108.801550
3,95476,SONOMA,38.291880,-122.457280
4,94559,NAPA,38.026885,-121.881924
5,94954,PETALUMA,38.232470,-122.644440
6,94571,RIO VISTA,38.155540,-121.690430
7,94535,TRAVIS AFB,38.271530,-121.933030
8,94503,AMERICAN CANYON,38.168050,-122.252770
9,94949,NOVATO,38.106090,-122.567900


### Creating a map of Borough's

In [6]:
df_raw2_map= folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng,  borough in zip(df_raw2['Latitude'], df_raw2['Longitude'], df_raw2['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(df_raw2_map)  
    
df_raw2_map

### Utilizing the Foursquare API to explore the neighborhoods and segment them

In [7]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'FMXFWWVAYKT0M3I5P4DKN3RVZF4SIJXCK45ZNMWCZZ3XDWOK' # your Foursquare ID
CLIENT_SECRET = '52UC2HJC2IWZYWHK0XH0NELJKCUM4VSADLYA0BQRERECB3M3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [8]:
# Get data of first neighborhood
borough_latitude = df_raw2['Latitude'][0] 
borough_longitude = df_raw2['Longitude'][0] 

borough_name = df_raw2['Borough'][0] 

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of NAPA are 38.02688500000005, -121.88192399999997.


### Find near by venues

In [9]:
# Setup API URL to explore venues near by
LIMIT = 100
radius = 1000
url = ('https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'
       .format(CLIENT_ID, CLIENT_SECRET, borough_latitude, borough_longitude, VERSION, radius, LIMIT))
results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Steeltown Coffee & Tea,Café,38.030414,-121.884043
1,E.J. Phair Brewing Co.,Brewery,38.033092,-121.882404
2,Lumpy's Diner,Diner,38.031060,-121.883959
3,Pittsburg Farmers' Market,Farmers Market,38.031399,-121.884030
4,New Mecca Cafe,Mexican Restaurant,38.033207,-121.883478
5,Laveranda Ristorante Italiano,Italian Restaurant,38.032210,-121.883890
6,CreAsian,Vietnamese Restaurant,38.031041,-121.884045
7,Pittsburg City Park,Park,38.022315,-121.889527
8,Dad's BBQ,BBQ Joint,38.029681,-121.884373
9,Sassy Sweets,Dessert Shop,38.029886,-121.884450


### Creating a function to repeat the same process to all boroughs

In [10]:
def getNearbyVenues(names, latitudes, longitudes):
    
    LIMIT=100
    radius=1000
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
bayarea_venues_df = getNearbyVenues(names=df_raw2['Borough'],
                                   latitudes=df_raw2['Latitude'],
                                   longitudes=df_raw2['Longitude']
                                  )

NAPA
FAIRFIELD
DIXON
SONOMA
NAPA
PETALUMA
RIO VISTA
TRAVIS AFB
AMERICAN CANYON
NOVATO
NOVATO
BIRDS LANDING
VALLEJO
BENICIA
VALLEJO
VALLEJO
NOVATO
VALLEJO
NICASIO
OAKLEY
CROCKETT
PORT COSTA
SUISUN CITY
SAN FRANCISCO
PITTSBURG
SAN RAFAEL
CONCORD
RODEO
MARTINEZ
HERCULES
SAN GERONIMO
LAGUNITAS
ALAMEDA
ANTIOCH
SAN ANSELMO
BRENTWOOD
SAN FRANCISCO
CONCORD
FAIRFAX
WOODACRE
FOREST KNOLLS
WALNUT CREEK
PINOLE
RICHMOND
CONCORD
SAN PABLO
SAN RAFAEL
ANTIOCH
EL SOBRANTE
OAKLAND
PLEASANT HILL
CONCORD
GREENBRAE
SAN FRANCISCO
LAFAYETTE
CLAYTON
RICHMOND
RICHMOND
LARKSPUR
SAN QUENTIN
EL CERRITO
CORTE MADERA
WALNUT CREEK
BERKELEY
SAN FRANCISCO
MILL VALLEY
ORINDA
BERKELEY
BERKELEY
BYRON
STINSON BEACH
ALBANY
BERKELEY
SAN FRANCISCO
WALNUT CREEK
BERKELEY
BERKELEY
BERKELEY
ALAMO
BERKELEY
SAUSALITO
MORAGA
BELVEDERE TIBURON
SAN FRANCISCO
BERKELEY
OAKLAND
OAKLAND
OAKLAND
LIVERMORE
EMERYVILLE
DIABLO
DANVILLE
DANVILLE
SAN FRANCISCO
OAKLAND
SAN FRANCISCO
OAKLAND
SAN RAMON
OAKLAND
OAKLAND
CASTRO VALLEY
SAN FRANCISCO
S

### Some rows form resulting dataframe with venues

In [12]:
bayarea_venues_df.head(12)

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,NAPA,38.026885,-121.881924,Steeltown Coffee & Tea,38.030414,-121.884043,Café
1,NAPA,38.026885,-121.881924,E.J. Phair Brewing Co.,38.033092,-121.882404,Brewery
2,NAPA,38.026885,-121.881924,Lumpy's Diner,38.031060,-121.883959,Diner
3,NAPA,38.026885,-121.881924,Pittsburg Farmers' Market,38.031399,-121.884030,Farmers Market
4,NAPA,38.026885,-121.881924,New Mecca Cafe,38.033207,-121.883478,Mexican Restaurant
5,NAPA,38.026885,-121.881924,Laveranda Ristorante Italiano,38.032210,-121.883890,Italian Restaurant
6,NAPA,38.026885,-121.881924,CreAsian,38.031041,-121.884045,Vietnamese Restaurant
7,NAPA,38.026885,-121.881924,Pittsburg City Park,38.022315,-121.889527,Park
8,NAPA,38.026885,-121.881924,Dad's BBQ,38.029681,-121.884373,BBQ Joint
9,NAPA,38.026885,-121.881924,Sassy Sweets,38.029886,-121.884450,Dessert Shop


### Checking how many venues were returned for each botough

In [13]:
bayarea_venues_df.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
ALAMEDA,32,32,32,32,32,32
ALAMO,31,31,31,31,31,31
ALBANY,100,100,100,100,100,100
ALVISO,22,22,22,22,22,22
AMERICAN CANYON,40,40,40,40,40,40
ANTIOCH,32,32,32,32,32,32
ATHERTON,10,10,10,10,10,10
BELMONT,37,37,37,37,37,37
BELVEDERE TIBURON,37,37,37,37,37,37


### Let's find out how many unique categories can be curated from all the returned venues

In [14]:
print('There are {} uniques categories.'.format(len(bayarea_venues_df['Venue Category'].unique())))

There are 353 uniques categories.


### Find a short-list of boroughs with restauraunts

In [15]:
print('Boroughs with restaurants and cafés:')
restau_cafe = bayarea_venues_df['Venue Category'].str.contains('Restaurant|Bar|Café|Pub|Cuisine', case=False, regex=True)
bayarea_venues_toeat = bayarea_venues_df[restau_cafe].reset_index(drop=True)

bayarea_venues_toeat.head(12)

Boroughs with restaurants and cafés:


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,NAPA,38.026885,-121.881924,Steeltown Coffee & Tea,38.030414,-121.884043,Café
1,NAPA,38.026885,-121.881924,New Mecca Cafe,38.033207,-121.883478,Mexican Restaurant
2,NAPA,38.026885,-121.881924,Laveranda Ristorante Italiano,38.032210,-121.883890,Italian Restaurant
3,NAPA,38.026885,-121.881924,CreAsian,38.031041,-121.884045,Vietnamese Restaurant
4,NAPA,38.026885,-121.881924,Skyview Noodle & Tea,38.032825,-121.882040,Asian Restaurant
5,NAPA,38.026885,-121.881924,La Auora,38.029298,-121.890066,Mexican Restaurant
6,NAPA,38.026885,-121.881924,Waterfront Grill & Cafe,38.035077,-121.883607,American Restaurant
7,NAPA,38.026885,-121.881924,KiteCafe,38.035492,-121.883458,Café
8,FAIRFIELD,38.249240,-122.044300,Chez Soul,38.249759,-122.046145,Southern / Soul Food Restaurant
9,FAIRFIELD,38.249240,-122.044300,China Palace,38.249492,-122.045125,Chinese Restaurant


In [16]:
bayarea_venues_toeat_count = bayarea_venues_toeat.count()
bayarea_venues_toeat_count

Borough              2273
Borough Latitude     2273
Borough Longitude    2273
Venue                2273
Venue Latitude       2273
Venue Longitude      2273
Venue Category       2273
dtype: int64

In [17]:
print('Boroughs with Indian restaurants and café:')
ind_restau_cafe = bayarea_venues_toeat['Venue Category'].str.contains('Indian', case=False, regex=True)
bayarea_venues_toeat_ind = bayarea_venues_toeat[ind_restau_cafe].drop_duplicates().reset_index(drop=True)

bayarea_venues_toeat_ind

Boroughs with Indian restaurants and café:


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,FAIRFIELD,38.249240,-122.044300,Saffron Indian Cuisine & Bar,38.249067,-122.042989,Indian Restaurant
1,SONOMA,38.291880,-122.457280,Taste of the Himalayas,38.292343,-122.455789,Indian Restaurant
2,SONOMA,38.291880,-122.457280,delhi belly,38.291558,-122.457757,North Indian Restaurant
3,AMERICAN CANYON,38.168050,-122.252770,All Spice Indian Restaurant,38.166265,-122.254110,Indian Restaurant
4,NOVATO,38.106090,-122.567900,Batika India Bistro,38.107328,-122.568293,Indian Restaurant
5,BENICIA,38.052850,-122.153510,Aroma Indian Cuisine,38.050871,-122.157720,Indian Restaurant
6,CONCORD,37.981650,-122.025830,Naan n Curry,37.977223,-122.035039,Indian Restaurant
7,LAGUNITAS,38.014210,-122.696570,Arti Cafe,38.012457,-122.701456,Indian Restaurant
8,FAIRFAX,37.985170,-122.590380,Cafe Lotus,37.987427,-122.588553,Indian Restaurant
9,FAIRFAX,37.985170,-122.590380,Avatars Punjabi Burrito,37.986032,-122.583616,Indian Restaurant


In [18]:
print('Top 25 Boroughs with restaurants and cafés:')
bayarea_venues_toeat_top25 = (bayarea_venues_df[restau_cafe]
                      .groupby(['Borough'])
                      .count()['Venue']
                      .sort_values(ascending=False)
                      [:25]
                     )
bayarea_venues_toeat_top25

Top 25 Boroughs with restaurants and cafés:


Borough
HAYWARD          180
REDWOOD CITY     176
SUNNYVALE        160
FREMONT          124
WALNUT CREEK     117
SAN FRANCISCO     81
NOVATO            78
MOUNTAIN VIEW     72
PLEASANTON        66
CONCORD           64
PALO ALTO         60
SAN MATEO         60
SAN JOSE          53
CASTRO VALLEY     48
SAN CARLOS        46
LOS ALTOS         44
ALBANY            39
DALY CITY         38
SAN BRUNO         36
MENLO PARK        34
MILLBRAE          32
LAFAYETTE         32
SONOMA            30
PLEASANT HILL     30
VALLEJO           28
Name: Venue, dtype: int64

### A short-list of Boroughs where there are a lot of restauraunts and no Indian restauraunts

In [19]:
bayarea_venues_toeat_top8 = bayarea_venues_toeat_top25.drop(labels=['SONOMA', 'NOVATO', 'CONCORD', 'WALNUT CREEK', 'LAFAYETTE',
                                                                     'ALBANY', 'CASTRO VALLEY', 'PLEASANTON', 'HAYWARD', 'SAN BRUNO',
                                                                     'MILLBRAE', 'FREMONT', 'REDWOOD CITY', 'SAN CARLOS', 'SAN JOSE',
                                                                     'SUNNYVALE', 'MOUNTAIN VIEW'])
bayarea_venues_toeat_top8

Borough
SAN FRANCISCO    81
PALO ALTO        60
SAN MATEO        60
LOS ALTOS        44
DALY CITY        38
MENLO PARK       34
PLEASANT HILL    30
VALLEJO          28
Name: Venue, dtype: int64

### Encoding venues

In [20]:
# one hot encoding
bayarea_onehot = pd.get_dummies(bayarea_venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bayarea_onehot['Borough'] = bayarea_venues_df['Borough'] 

# move neighborhood column to the first column
fixed_columns = [bayarea_onehot.columns[-1]] + list(bayarea_onehot.columns[:-1])
bayarea_onehot = bayarea_onehot[fixed_columns]

bayarea_onehot.head()

,Borough,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Campground,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,City,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Basketball Court,College Cafeteria,College Football Field,College Rec Center,College Soccer Field,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sca

In [21]:
bayarea_onehot.shape

(7129, 354)

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
bayarea_venues_grouped = bayarea_onehot.groupby('Borough').mean().reset_index()
bayarea_venues_grouped

,Borough,ATM,Accessories Store,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Campground,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,City,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Basketball Court,College Cafeteria,College Football Field,College Rec Center,College Soccer Field,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gelato Shop,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Mongolian Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Nature Preserve,Neighborhood,New American Restaurant,Night Market,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Restaurant,River,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Sca

### Each borough along with the top 5 most common venues

In [23]:
num_top_venues = 5

for hood in bayarea_venues_grouped['Borough']:
    print("----"+hood+"----")
    temp = bayarea_venues_grouped[bayarea_venues_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALAMEDA----
                  venue  freq
0                 Beach  0.12
1           Coffee Shop  0.12
2   Arts & Crafts Store  0.06
3  Gym / Fitness Center  0.06
4         Grocery Store  0.06


----ALAMO----
                 venue  freq
0             Pharmacy  0.06
1  American Restaurant  0.06
2          Coffee Shop  0.06
3          Pizza Place  0.06
4            Pet Store  0.03


----ALBANY----
                 venue  freq
0          Coffee Shop  0.06
1      Thai Restaurant  0.06
2    Indian Restaurant  0.04
3  Sporting Goods Shop  0.04
4          Pizza Place  0.04


----ALVISO----
                venue  freq
0          Food Truck  0.18
1               Hotel  0.14
2  Mexican Restaurant  0.14
3   Convenience Store  0.05
4                 Gym  0.05


----AMERICAN CANYON----
                venue  freq
0                Park  0.08
1                 ATM  0.05
2  Chinese Restaurant  0.05
3            Pharmacy  0.05
4   Convenience Store  0.05


----ANTIOCH----
                venue  fre

### A function to sort the venues in descending order

In [24]:
# method to sort venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create the new dataframe and display the top 10 venues for each borough

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = bayarea_venues_grouped['Borough']

for ind in np.arange(bayarea_venues_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bayarea_venues_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALAMEDA,Coffee Shop,Beach,Pool,Skating Rink,Gastropub,Grocery Store,Marijuana Dispensary,Gym / Fitness Center,Arts & Crafts Store,Harbor / Marina
1,ALAMO,Coffee Shop,Pharmacy,Pizza Place,American Restaurant,Sandwich Place,Bank,Fast Food Restaurant,New American Restaurant,Gas Station,Chinese Restaurant
2,ALBANY,Thai Restaurant,Coffee Shop,Pizza Place,Sporting Goods Shop,Indian Restaurant,Pet Store,Mexican Restaurant,Café,Bar,Brewery
3,ALVISO,Food Truck,Hotel,Mexican Restaurant,Hotel Bar,Coffee Shop,Bar,Electronics Store,Café,Gym,Trail
4,AMERICAN CANYON,Park,ATM,Pharmacy,Pizza Place,Chinese Restaurant,Shopping Mall,Convenience Store,Hotel,Brewery,Thrift / Vintage Store


### Run k-means to cluster the neighborhood into 5 clusters

In [26]:
# set number of clusters
kclusters = 5

bayarea_venues_grouped_clustering = bayarea_venues_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(bayarea_venues_grouped_clustering)

# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bayarea_venues_merged = df_raw2.copy()
bayarea_venues_merged.rename(columns={'Borough':'Borough'}, inplace=True)
bayarea_venues_merged = bayarea_venues_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

bayarea_venues_merged.head() # check the last columns!

,ZIP,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94558,NAPA,38.026885,-121.881924,0.0,Café,Mexican Restaurant,Construction & Landscaping,Farmers Market,Liquor Store,BBQ Joint,Toy / Game Store,Diner,Park,Dessert Shop
1,94533,FAIRFIELD,38.249240,-122.044300,0.0,Mexican Restaurant,Theater,Coffee Shop,Chinese Restaurant,Hotel,American Restaurant,Burger Joint,Ice Cream Shop,Food & Drink Shop,Korean Restaurant
2,95620,DIXON,59.383710,-108.801550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,95476,SONOMA,38.291880,-122.457280,0.0,Wine Bar,Hotel,American Restaurant,Coffee Shop,Pizza Place,Bed & Breakfast,Portuguese Restaurant,Bank,Bar,Italian Restaurant
4,94559,NAPA,38.026885,-121.881924,0.0,Café,Mexican Restaurant,Construction & Landscaping,Farmers Market,Liquor Store,BBQ Joint,Toy / Game Store,Diner,Park,Dessert Shop


In [27]:
bayarea_venues_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)
bayarea_venues_merged.head()

,ZIP,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94558,NAPA,38.026885,-121.881924,0.0,Café,Mexican Restaurant,Construction & Landscaping,Farmers Market,Liquor Store,BBQ Joint,Toy / Game Store,Diner,Park,Dessert Shop
1,94533,FAIRFIELD,38.249240,-122.044300,0.0,Mexican Restaurant,Theater,Coffee Shop,Chinese Restaurant,Hotel,American Restaurant,Burger Joint,Ice Cream Shop,Food & Drink Shop,Korean Restaurant
3,95476,SONOMA,38.291880,-122.457280,0.0,Wine Bar,Hotel,American Restaurant,Coffee Shop,Pizza Place,Bed & Breakfast,Portuguese Restaurant,Bank,Bar,Italian Restaurant
4,94559,NAPA,38.026885,-121.881924,0.0,Café,Mexican Restaurant,Construction & Landscaping,Farmers Market,Liquor Store,BBQ Joint,Toy / Game Store,Diner,Park,Dessert Shop
5,94954,PETALUMA,38.232470,-122.644440,0.0,Pizza Place,Italian Restaurant,Coffee Shop,Pub,Bar,Brewery,Grocery Store,Bakery,Ice Cream Shop,Thai Restaurant


### Visualize the resulting clusters

In [28]:
# create map
map_clusters = folium.Map(location=[df_raw2["Latitude"][0], df_raw2["Longitude"][0]], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bayarea_venues_merged['Latitude'], bayarea_venues_merged['Longitude'], bayarea_venues_merged['Borough'], bayarea_venues_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

### Cluster 1

In [29]:
bayarea_venues_merged.loc[bayarea_venues_merged['Cluster Labels'] == 0, bayarea_venues_merged.columns[[1] + list(range(5, bayarea_venues_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NAPA,Café,Mexican Restaurant,Construction & Landscaping,Farmers Market,Liquor Store,BBQ Joint,Toy / Game Store,Diner,Park,Dessert Shop
1,FAIRFIELD,Mexican Restaurant,Theater,Coffee Shop,Chinese Restaurant,Hotel,American Restaurant,Burger Joint,Ice Cream Shop,Food & Drink Shop,Korean Restaurant
3,SONOMA,Wine Bar,Hotel,American Restaurant,Coffee Shop,Pizza Place,Bed & Breakfast,Portuguese Restaurant,Bank,Bar,Italian Restaurant
4,NAPA,Café,Mexican Restaurant,Construction & Landscaping,Farmers Market,Liquor Store,BBQ Joint,Toy / Game Store,Diner,Park,Dessert Shop
5,PETALUMA,Pizza Place,Italian Restaurant,Coffee Shop,Pub,Bar,Brewery,Grocery Store,Bakery,Ice Cream Shop,Thai Restaurant
6,RIO VISTA,American Restaurant,Harbor / Marina,Grocery Store,Post Office,Gym,Pizza Place,Cocktail Bar,River,Sandwich Place,Chinese Restaurant
7,TRAVIS AFB,Playground,Bowling Alley,Convenience Store,Airport Terminal,Hotel,Automotive Shop,Café,Fried Chicken Joint,Cafeteria,Bar
8,AMERICAN CANYON,Park,ATM,Pharmacy,Pizza Place,Chinese Restaurant,Shopping Mall,Convenience Store,Hotel,Brewery,Thrift / Vintage Store
9,NOVATO,Pizza Place,Mexican Restaurant,Italian Restaurant,Coffee Shop,Grocery Store,Chinese Restaurant,Bank,Bakery,Breakfast Spot,Sandwich Place
10,NOVATO,Pizza Place,Mexican Restaurant,Italian Restaurant,Coffee Shop,Grocery Store,Chinese Restaurant,Bank,Bakery,Breakfast Spot,Sandwich Place


### Cluster 2

In [30]:
bayarea_venues_merged.loc[bayarea_venues_merged['Cluster Labels'] == 1, bayarea_venues_merged.columns[[1] + list(range(5, bayarea_venues_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,BIRDS LANDING,Bar,Business Service,Zoo Exhibit,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Fair
90,DIABLO,Restaurant,Baseball Field,Golf Course,Falafel Restaurant,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop
129,DUBLIN,Restaurant,Pharmacy,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop


### Cluster 3

In [31]:
bayarea_venues_merged.loc[bayarea_venues_merged['Cluster Labels'] == 2, bayarea_venues_merged.columns[[1] + list(range(5, bayarea_venues_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
173,MOUNT HAMILTON,Campground,Zoo Exhibit,Concert Hall,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Fair


### Cluster 4

In [32]:
bayarea_venues_merged.loc[bayarea_venues_merged['Cluster Labels'] == 3, bayarea_venues_merged.columns[[1] + list(range(5, bayarea_venues_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
148,HALF MOON BAY,Boat or Ferry,Zoo Exhibit,Farm,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space,Fabric Shop,Fair


### Cluster 5

In [33]:
bayarea_venues_merged.loc[bayarea_venues_merged['Cluster Labels'] == 4, bayarea_venues_merged.columns[[1] + list(range(5, bayarea_venues_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space
85,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space
86,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space
87,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space
94,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space
96,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space
98,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space
99,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space
103,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space
105,OAKLAND,Park,Intersection,Zoo Exhibit,Falafel Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Event Space


## Results
The following are the main outcomes of the project work:

i. DALY CITY, MENLO PARK and PLEASANT HILL are the best boroughs for a new Indian restaurant business in the San Francisco Bay Area.

ii. DALY CITY, MENLO PARK and PLEASANT HILL boroughs are similar to boroughs where existing Indian restaurants are located.

## Discussion
The main results are presented in the previous section. We can recommend DALY CITY, MENLO PARK and PLEASANT HILL as the best boroughs to open an Indian restaurant. These boroughs have a lot other restaurants, cafes, and shopping malls what can be considered as ‘a center of attraction’. At the same time these boroughs don’t have direct competitors (Indian restaurants), which is important for business development.

We should avoid boroughs such as FAIRFIELD, AMERICAN CANYON, BENICIA, LAGUNITAS, FAIRFAX, EL CERRITO, MILL VALLEY, SAUSALITO, EMERYVILLE, SAN RAMON, SAN LORENZO, UNION CITY, BURLINGAME, NEWARK, MILPITAS, SONOMA, NOVATO, CONCORD, WALNUT CREEK, LAFAYETTE, ALBANY, CASTRO VALLEY, PLEASANTON, HAYWARD, SAN BRUNO, MILLBRAE, FREMONT, REDWOOD CITY, SAN CARLOS, SAN JOSE, SUNNYVALE, MOUNTAIN VIEW etc. to run an Indian restaurant because they have already Indian restaurants and these restaurants are successfully running the business.

## Conclusion
The main aim of the project is to identify the best borough in San Francisco Bay Area to start an Indian restaurant business. In order to find the best borough, we use criteria’s such as (i) “a center of attraction” for dining, (ii) minimum Indian restaurants, and (iii) k-means clustering to find similar boroughs to the boroughs where Indian restaurants are located. As a result, we can recommend DALY CITY, MENLO PARK and PLEASANT HILL are the best boroughs to open an Indian restaurant. For the further improvement of the project, it is a good idea to consider a large radius value i.e. larger than 1000 m and collect more information like income, population, rent payments, crime level, etc. It can help to cover some more good places within the boroughs and predict more accurate results.